## Imports

In [2]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from helper_functions import *

## Loading data

In [ ]:
data_folder = "raw_data"

first_ecg = np.load(data_folder + "/first_ecg.npy")
first_ppg = np.load(data_folder + "/first_ppg.npy")

second_ecg = np.load(data_folder + "/second_ecg.npy")
second_ppg = np.load(data_folder + "/second_ppg.npy")

third_ecg = np.load(data_folder + "/third_ecg.npy")
third_ppg = np.load(data_folder + "/third_ppg.npy")

## 1. Sampling frequencies

In [ ]:
first_ecg_ma_detrended = simple_ma_detrending(first_ecg)
first_ppg_ma_detrended = simple_ma_detrending(first_ppg)

second_ecg_ma_detrended = simple_ma_detrending(second_ecg)
second_ppg_ma_detrended = simple_ma_detrending(second_ppg)

third_ecg_ma_detrended = simple_ma_detrending(third_ecg)
third_ppg_ma_detrended = simple_ma_detrending(third_ppg)


In [ ]:
first_ecg_freq = find_frequency(first_ecg_ma_detrended, threshold)